In [108]:
import pandas as pd
import numpy as np
from numpy import nan as NA
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

## Indexação hierárquica
<br>
A indexação hierárquica nos permite ter dois ou mais níveis de indíces em um eixo. Facilitando o trabalho com dados de dimensões mais altas em um formato de dimensões menores.

In [109]:
data = pd.Series(np.random.randn(9), index=[['a','a','a','b','b','c','c','d','d'],
                                            [1,2,3,1,3,1,2,2,3]])
data
print('-----------')
data.index # obtendo as divisões MultiIndex de uma Series


a  1   -0.268600
   2    2.233848
   3    1.112276
b  1    0.374500
   3    1.382161
c  1   -0.550821
   2   -0.249719
d  2    0.449906
   3   -0.258428
dtype: float64

-----------


MultiIndex([('a', 1),
            ('a', 2),
            ('a', 3),
            ('b', 1),
            ('b', 3),
            ('c', 1),
            ('c', 2),
            ('d', 2),
            ('d', 3)],
           )

In [110]:
# fatiando uma indexação hierárquica
data['b']
print('-----------')
data['b':'c']
print('-----------')
data.loc[['b','d']]

1    0.374500
3    1.382161
dtype: float64

-----------


b  1    0.374500
   3    1.382161
c  1   -0.550821
   2   -0.249719
dtype: float64

-----------


b  1    0.374500
   3    1.382161
d  2    0.449906
   3   -0.258428
dtype: float64

In [111]:
# fatiando e selecionando o nível
data.loc[:,2] # todos os níveis externos que tiverem nível interno =  2

a    2.233848
c   -0.249719
d    0.449906
dtype: float64

In [112]:
# reorganizando os dados em um DataFrame com método unstack
data.unstack()
print('-----------')
data.unstack().stack()

,1,2,3
a,-0.268600,2.233848,1.112276
b,0.374500,NaN,1.382161
c,-0.550821,-0.249719,NaN
d,NaN,0.449906,-0.258428


-----------


a  1   -0.268600
   2    2.233848
   3    1.112276
b  1    0.374500
   3    1.382161
c  1   -0.550821
   2   -0.249719
d  2    0.449906
   3   -0.258428
dtype: float64

In [113]:
# DataFrames podem ter hieráquia em qualquer eixo
frame = pd.DataFrame(np.arange(12).reshape((4,3)), index=[['a','a','b','b'],[1,2,1,2]],
                    columns=[['Ohio','Ohio','Colorado'], ['Green','Red','Green']])
frame
print('----------')
# podemos dar nomes aos níveis hieráquicos de um DataFrame
frame.index.names=['key1','key2']
frame.columns.names=['state','color']
frame

Ohio     Colorado
    Green Red    Green
a 1     0   1        2
  2     3   4        5
b 1     6   7        8
  2     9  10       11

----------


state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
     2        3   4        5
b    1        6   7        8
     2        9  10       11

In [114]:
# indexação de um DataFrame hierárquico
frame['Ohio']

color      Green  Red
key1 key2            
a    1         0    1
     2         3    4
b    1         6    7
     2         9   10

In [115]:
# podemos criar um MultiIndex de maneira independente
pd.MultiIndex.from_arrays([['Ohio','Ohio','Colorad'],['Green','Red','Green']], names=['state','color'])

MultiIndex([(   'Ohio', 'Green'),
            (   'Ohio',   'Red'),
            ('Colorad', 'Green')],
           names=['state', 'color'])

### Reorganizando e ordenando níveis

In [116]:
# trocando eixos de níveis
frame.swaplevel('key1','key2')
print('-------------------')
# ordenando dados usando os valores de um só indíce
frame.sort_index(level = 1)


state      Ohio     Colorado
color     Green Red    Green
key2 key1                   
1    a        0   1        2
2    a        3   4        5
1    b        6   7        8
2    b        9  10       11

-------------------


state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
b    1        6   7        8
a    2        3   4        5
b    2        9  10       11

In [117]:
# trocando eixos de níveis e ordenando-os
frame.swaplevel(0,1).sort_index(level=1)

state      Ohio     Colorado
color     Green Red    Green
key2 key1                   
1    a        0   1        2
2    a        3   4        5
1    b        6   7        8
2    b        9  10       11

### Estatísticas de reusmo por nível

In [118]:
# Somando os valores de um nível
frame.sum(level='key2')
# Somando os valores de um nível em um eixo específico
print('--------------')
frame.sum(level='color',axis=1)

state  Ohio     Colorado
color Green Red    Green
key2                    
1         6   8       10
2        12  14       16

--------------


color      Green  Red
key1 key2            
a    1         2    1
     2         8    4
b    1        14    7
     2        20   10

### Indexando colunas de um DataFrame

In [119]:
frame = pd.DataFrame({'a':range(7),'b':range(7,0,-1), 
                      'c':['one','one','one','two','two','two','two'],
                      'd':[0,1,2,0,1,2,3]})
frame

print('------------------------------')
# usaremos colunas como index
frame2 = frame.set_index(['c','d'])
frame2

# passando valores que são iguais aos das colunas sem tirá-los das colunas
print('------------------------------')
frame.set_index(['c','d'],drop=False)

print('------------------------------')
# passando níveis hierárquicos para colunas
frame2.reset_index()

,a,b,c,d
0,0,7,one,0
1,1,6,one,1
2,2,5,one,2
3,3,4,two,0
4,4,3,two,1
5,5,2,two,2
6,6,1,two,3


------------------------------


a  b
c   d      
one 0  0  7
    1  1  6
    2  2  5
two 0  3  4
    1  4  3
    2  5  2
    3  6  1

------------------------------


a  b    c  d
c   d              
one 0  0  7  one  0
    1  1  6  one  1
    2  2  5  one  2
two 0  3  4  two  0
    1  4  3  two  1
    2  5  2  two  2
    3  6  1  two  3

------------------------------


,c,d,a,b
0,one,0,0,7
1,one,1,1,6
2,one,2,2,5
3,two,0,3,4
4,two,1,4,3
5,two,2,5,2
6,two,3,6,1


## Combinando e mesclando conjuntos de dados

pandas.merge(): conecta linha em DataFrames (como joins em SQL).
<br>
pandas.concat(): concatena objetos ao longo de um eixo.
<br>
combine_first(): combina dados que se sobrepõe.

### Junções no DataFrame

In [120]:
# criando os DataFrames
df1 = pd.DataFrame({'key':['b','b','a','c','a','a','b'], 'data1': range(7)})
df2 = pd.DataFrame({'key':['a','b','d'], 'data2': range(3)})

# output dos DataFrames
df1
print('------------------------------------------------')
df2

,key,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,a,5
6,b,6


------------------------------------------------


,key,data2
0,a,0
1,b,1
2,d,2


In [121]:
# método merge()
pd.merge(df1,df2)

# método merge() especificando a coluna para se fazer a junção (boa prática)
pd.merge(df1,df2, on='key')

,key,data1,data2
0,b,0,1
1,b,1,1
2,b,6,1
3,a,2,0
4,a,4,0
5,a,5,0


,key,data1,data2
0,b,0,1
1,b,1,1
2,b,6,1
3,a,2,0
4,a,4,0
5,a,5,0


In [122]:
# criando um DataFrame
df3 = pd.DataFrame({'lkey':['b','b','a','c','a','a','b'], 'data1': range(7)})
df4 = pd.DataFrame({'rkey': ['a','b','d'],'data2': range(3)})

df3
print('-----------------------------')
df4

,lkey,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,a,5
6,b,6


-----------------------------


,rkey,data2
0,a,0
1,b,1
2,d,2


In [123]:
# método merge() unindo lateralmente
pd.merge(df3, df4, left_on='lkey',right_on='rkey')

,lkey,data1,rkey,data2
0,b,0,b,1
1,b,1,b,1
2,b,6,b,1
3,a,2,a,0
4,a,4,a,0
5,a,5,a,0


In [124]:
# método merge() em outer
pd.merge(df1,df2, how='outer')

,key,data1,data2
0,b,0.0,1.0
1,b,1.0,1.0
2,b,6.0,1.0
3,a,2.0,0.0
4,a,4.0,0.0
5,a,5.0,0.0
6,c,3.0,NaN
7,d,NaN,2.0


##### Diferentes tipos de junções com argumento how
<br>
'inner': somente combinações de chaves observadas nas duas tabelas.
<br>
'left': todas as combinações de chaves encontradas na tabela à esquerda.
<br>
'right': todas as combinações de chaves encontradas na tabela à direita
<br>
'outer': todas as combinações de chaves observadas em ambas tabelas em conjunto

In [125]:
df1 = pd.DataFrame({'key':['b','b','a','c','a','b'], 'data1': range(6)})
df2 = pd.DataFrame({'key':['a','b','a','b','d'], 'data2': range(5)})

df1
df2

,key,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,b,5


,key,data2
0,a,0
1,b,1
2,a,2
3,b,3
4,d,4


In [126]:
pd.merge(df1,df2, on='key', how='left')

,key,data1,data2
0,b,0,1.0
1,b,0,3.0
2,b,1,1.0
3,b,1,3.0
4,a,2,0.0
5,a,2,2.0
6,c,3,NaN
7,a,4,0.0
8,a,4,2.0
9,b,5,1.0


In [127]:
pd.merge(df1,df2, on='key', how = 'inner') #intersecção dos DataFrames

,key,data1,data2
0,b,0,1
1,b,0,3
2,b,1,1
3,b,1,3
4,b,5,1
5,b,5,3
6,a,2,0
7,a,2,2
8,a,4,0
9,a,4,2


In [128]:
# criando DataFrames
left = pd.DataFrame({'key1': ['foo','foo','bar'],'key2':['one','two','one'],'lval':[1,2,3]})
right = pd.DataFrame({'key1': ['foo','foo','bar','bar'],
                      'key2':['one','one','one','two'],
                      'rval':[4,5,6,7]})
left
right

,key1,key2,lval
0,foo,one,1
1,foo,two,2
2,bar,one,3


,key1,key2,rval
0,foo,one,4
1,foo,one,5
2,bar,one,6
3,bar,two,7


In [129]:
# união dos DataFrame
pd.merge(left, right, on=['key1','key2'], how='outer')

,key1,key2,lval,rval
0,foo,one,1.0,4.0
1,foo,one,1.0,5.0
2,foo,two,2.0,NaN
3,bar,one,3.0,6.0
4,bar,two,NaN,7.0


In [130]:
pd.merge(left, right, on='key1')

,key1,key2_x,lval,key2_y,rval
0,foo,one,1,one,4
1,foo,one,1,one,5
2,foo,two,2,one,4
3,foo,two,2,one,5
4,bar,one,3,one,6
5,bar,one,3,two,7


In [131]:
pd.merge(left, right, on='key1', suffixes=('_left','_right'))

,key1,key2_left,lval,key2_right,rval
0,foo,one,1,one,4
1,foo,one,1,one,5
2,foo,two,2,one,4
3,foo,two,2,one,5
4,bar,one,3,one,6
5,bar,one,3,two,7


### Merge com base no índice

In [132]:
left1 = pd.DataFrame({'key':['a','b','a','a','b','c'],'value':range(6)})
right1 = pd.DataFrame({'group_val':[3.5,7]}, index=['a','b'])
left1
right1

,key,value
0,a,0
1,b,1
2,a,2
3,a,3
4,b,4
5,c,5


,group_val
a,3.5
b,7.0


#### Merge entre index e key
<br>
Explicitamos com os argumentos left_on/right_on e left_index/right_index qual DF está com uma key e qual está indexado normalmente

In [133]:
pd.merge(left1, right1, left_on='key',right_index=True)

,key,value,group_val
0,a,0,3.5
2,a,2,3.5
3,a,3,3.5
1,b,1,7.0
4,b,4,7.0


In [134]:
pd.merge(left1, right1, left_on='key',right_index=True,how='outer')

,key,value,group_val
0,a,0,3.5
2,a,2,3.5
3,a,3,3.5
1,b,1,7.0
4,b,4,7.0
5,c,5,NaN


#### Merge em hireraquicamente indexados
<br>
Como possuem vários índices, especificamos várias colunas na forma de lista com base nas quais o merge será executado. 

In [135]:
lefth = pd.DataFrame({'key1': ['Ohio','Ohio','Ohio','Nevada','Nevada'],
                     'key2': [2000, 2001, 2002, 2001, 2002], 'data':np.arange(5.)})
righth = pd.DataFrame(np.arange(12).reshape((6,2)), 
                      index=[['Nevada','Nevada','Ohio','Ohio','Ohio','Ohio'],
                            [2001, 2000, 2000, 2000, 2001, 2002]],
                     columns=['event1','event2'])

lefth
righth

,key1,key2,data
0,Ohio,2000,0.0
1,Ohio,2001,1.0
2,Ohio,2002,2.0
3,Nevada,2001,3.0
4,Nevada,2002,4.0


event1  event2
Nevada 2001       0       1
       2000       2       3
Ohio   2000       4       5
       2000       6       7
       2001       8       9
       2002      10      11

In [136]:
pd.merge(lefth,righth,left_on=['key1','key2'],right_index=True)

,key1,key2,data,event1,event2
0,Ohio,2000,0.0,4,5
0,Ohio,2000,0.0,6,7
1,Ohio,2001,1.0,8,9
2,Ohio,2002,2.0,10,11
3,Nevada,2001,3.0,0,1


In [137]:
pd.merge(lefth, righth, left_on=['key1','key2'], right_index=True, how='outer')

,key1,key2,data,event1,event2
0,Ohio,2000,0.0,4.0,5.0
0,Ohio,2000,0.0,6.0,7.0
1,Ohio,2001,1.0,8.0,9.0
2,Ohio,2002,2.0,10.0,11.0
3,Nevada,2001,3.0,0.0,1.0
4,Nevada,2002,4.0,NaN,NaN
4,Nevada,2000,NaN,2.0,3.0


In [138]:
left2 = pd.DataFrame([[1.,2.],[3.,4.],[5.,6.]], index=['a','c','e'],columns=['Ohio','Nevada'])
right2 =pd.DataFrame([[7.,8.],[9.,10.],[11.,12.],[13,14]],
                     index=['b','c','d','e'], 
                     columns=['Missouri','Alabama'])
left2
right2

,Ohio,Nevada
a,1.0,2.0
c,3.0,4.0
e,5.0,6.0


,Missouri,Alabama
b,7.0,8.0
c,9.0,10.0
d,11.0,12.0
e,13.0,14.0


In [139]:
# merge em dois DFs com index declarado
pd.merge(left2,right2, how='outer',left_index=True, right_index=True)

,Ohio,Nevada,Missouri,Alabama
a,1.0,2.0,NaN,NaN
b,NaN,NaN,7.0,8.0
c,3.0,4.0,9.0,10.0
d,NaN,NaN,11.0,12.0
e,5.0,6.0,13.0,14.0


Em casos de índices semenlhantes podemos utilizar .join()

In [140]:
left2.join(right2, how='outer')

,Ohio,Nevada,Missouri,Alabama
a,1.0,2.0,NaN,NaN
b,NaN,NaN,7.0,8.0
c,3.0,4.0,9.0,10.0
d,NaN,NaN,11.0,12.0
e,5.0,6.0,13.0,14.0


### Concatenação ao longo de um eixo

#### Concatenação com numpy

In [142]:
arr = np.arange(12).reshape((3,4))
arr

# concatenando
np.concatenate([arr,arr],axis=1)

array([[ 0,  1,  2,  3],
       [ 4,  5,  6,  7],
       [ 8,  9, 10, 11]])

array([[ 0,  1,  2,  3,  0,  1,  2,  3],
       [ 4,  5,  6,  7,  4,  5,  6,  7],
       [ 8,  9, 10, 11,  8,  9, 10, 11]])

#### Concatenação em pandas

In [143]:
s1 = pd.Series([0,1], index=['a','b'])
s2 = pd.Series([2,3,4], index=['c','d','e'])
s3 = pd.Series([5,6], index=['f','g'])

s1
s2
s3

a    0
b    1
dtype: int64

c    2
d    3
e    4
dtype: int64

f    5
g    6
dtype: int64

In [144]:
pd.concat([s1,s2,s3])

a    0
b    1
c    2
d    3
e    4
f    5
g    6
dtype: int64

In [145]:
pd.concat([s1,s2,s3], axis = 1)

,0,1,2
a,0.0,NaN,NaN
b,1.0,NaN,NaN
c,NaN,2.0,NaN
d,NaN,3.0,NaN
e,NaN,4.0,NaN
f,NaN,NaN,5.0
g,NaN,NaN,6.0


In [148]:
s4 = pd.concat([s1,s3])
s4

# concatenação sobreposta
pd.concat([s1,s4], axis=1)

a    0
b    1
f    5
g    6
dtype: int64

,0,1
a,0.0,0
b,1.0,1
f,NaN,5
g,NaN,6


In [152]:
# concatenando intersecção
pd.concat([s1,s4], axis=1, join='inner')

,0,1
a,0,0
b,1,1


#### Concatenação e hieraquização

In [154]:
# hierarquizando a concatenação de Series
result = pd.concat([s1,s2,s3],keys = ['one','two','three'])
result

result.unstack()

one    a    0
       b    1
two    c    2
       d    3
       e    4
three  f    5
       g    6
dtype: int64

,a,b,c,d,e,f,g
one,0.0,1.0,NaN,NaN,NaN,NaN,NaN
two,NaN,NaN,2.0,3.0,4.0,NaN,NaN
three,NaN,NaN,NaN,NaN,NaN,5.0,6.0


In [164]:
# hierarquizando a concatenação de DataFrames
df1 = pd.DataFrame(np.arange(6).reshape(3,2))
df2 = pd.DataFrame(5 + np.arange(4).reshape(2,2), index=['a','c'], columns=['three','four'])
df1
df2

pd.concat([df1,df2],axis=1,keys=['level1','level2'])
# passando os níveis para outro eixo
pd.concat({'level1':df1,'level2':df2},axis=0)
# nomenado os níveis
pd.concat([df1,df2],axis=1,keys=['level1','level2'], names=['upper','low'])

,0,1
0,0,1
1,2,3
2,4,5


,three,four
a,5,6
c,7,8


level1      level2     
       0    1  three four
0    0.0  1.0    NaN  NaN
1    2.0  3.0    NaN  NaN
2    4.0  5.0    NaN  NaN
a    NaN  NaN    5.0  6.0
c    NaN  NaN    7.0  8.0

0    1  four  three
level1 0  0.0  1.0   NaN    NaN
       1  2.0  3.0   NaN    NaN
       2  4.0  5.0   NaN    NaN
level2 a  NaN  NaN   6.0    5.0
       c  NaN  NaN   8.0    7.0

upper level1      level2     
low        0    1  three four
0        0.0  1.0    NaN  NaN
1        2.0  3.0    NaN  NaN
2        4.0  5.0    NaN  NaN
a        NaN  NaN    5.0  6.0
c        NaN  NaN    7.0  8.0

#### Concatenação em DFs com índices de linha sem dado relevante

In [165]:
df1 = pd.DataFrame(np.random.rand(3,4), columns=['a','b','c','d'])
df2 = pd.DataFrame(np.random.rand(2,3), columns=['b','d','a'])
df1
df2

,a,b,c,d
0,0.880456,0.474796,0.434095,0.806536
1,0.861972,0.275426,0.390518,0.081976
2,0.838633,0.198529,0.831128,0.548022


,b,d,a
0,0.293754,0.086805,0.806365
1,0.608827,0.559306,0.392400


In [168]:
pd.concat([df1,df2])
pd.concat([df1,df2], ignore_index=True)

,a,b,c,d
0,0.880456,0.474796,0.434095,0.806536
1,0.861972,0.275426,0.390518,0.081976
2,0.838633,0.198529,0.831128,0.548022
0,0.806365,0.293754,NaN,0.086805
1,0.392400,0.608827,NaN,0.559306


,a,b,c,d
0,0.880456,0.474796,0.434095,0.806536
1,0.861972,0.275426,0.390518,0.081976
2,0.838633,0.198529,0.831128,0.548022
3,0.806365,0.293754,NaN,0.086805
4,0.392400,0.608827,NaN,0.559306


### Combinação com sobreposição de índices

In [169]:
a = pd.Series([NA, 2.5, 0.0, 3.5, 4.5, NA], index=['f','e','d','c','b','a'])
b = pd.Series([0., NA, 2., NA, NA, 5.], index=['a','b','c','d','e','f'])
a
b

f    NaN
e    2.5
d    0.0
c    3.5
b    4.5
a    NaN
dtype: float64

a    0.0
b    NaN
c    2.0
d    NaN
e    NaN
f    5.0
dtype: float64

In [170]:
np.where(pd.isnull(a), b, a)

array([0. , 2.5, 0. , 3.5, 4.5, 5. ])

In [171]:
#inserindo valores de um DF onde há NAs em outro DF
b.combine_first(a)

a    0.0
b    4.5
c    2.0
d    0.0
e    2.5
f    5.0
dtype: float64